# NCEP explore

In [36]:
# imports
import os 
import datetime
import pandas

import iris
import cf_units

from oceanpy.sst import io as sst_io
from oceanpy.sst import climate as sst_climate

# Load

In [2]:
path = '/home/xavier/Projects/Oceanography/data/NCEP-DOE/Z500'

In [4]:
dfile = os.path.join(path, 'NCEP-DOE_z500.nc')

In [5]:
cubes = iris.load_cubes(dfile)

/home/xavier/anaconda3/lib/python3.7/site-packages/iris/fileformats/netcdf.py:395: UserWarning: WARNING: missing_value not used since it
cannot be safely cast to variable data type
  var = variable[keys]


In [6]:
cubes

[<iris 'Cube' of Daily mean Geopotential Height at 500mb / (m) (-- : 15157; latitude: 73; longitude: 144)>]

# Explore

In [7]:
cube = cubes[0]
cube

<iris 'Cube' of Daily mean Geopotential Height at 500mb / (m) (-- : 15157; latitude: 73; longitude: 144)>

In [8]:
cube.data.shape

/home/xavier/anaconda3/lib/python3.7/site-packages/iris/fileformats/netcdf.py:395: UserWarning: WARNING: missing_value not used since it
cannot be safely cast to variable data type
  var = variable[keys]


(15157, 73, 144)

In [9]:
cube.coords

<bound method Cube.coords of <iris 'Cube' of Daily mean Geopotential Height at 500mb / (m) (-- : 15157; latitude: 73; longitude: 144)>>

# Add time coord

## Load SST for reference

In [22]:
noaa_path = os.getenv('NOAA_OI')
SST = iris.load(os.path.join(noaa_path, 'sst.day.mean.1985.nc'))[0]

In [23]:
SST.coord('time')

DimCoord(array([67570., 67571., 67572., 67573., 67574., 67575., 67576., 67577.,
       67578., 67579., 67580., 67581., 67582., 67583., 67584., 67585.,
       67586., 67587., 67588., 67589., 67590., 67591., 67592., 67593.,
       67594., 67595., 67596., 67597., 67598., 67599., 67600., 67601.,
       67602., 67603., 67604., 67605., 67606., 67607., 67608., 67609.,
       67610., 67611., 67612., 67613., 67614., 67615., 67616., 67617.,
       67618., 67619., 67620., 67621., 67622., 67623., 67624., 67625.,
       67626., 67627., 67628., 67629., 67630., 67631., 67632., 67633.,
       67634., 67635., 67636., 67637., 67638., 67639., 67640., 67641.,
       67642., 67643., 67644., 67645., 67646., 67647., 67648., 67649.,
       67650., 67651., 67652., 67653., 67654., 67655., 67656., 67657.,
       67658., 67659., 67660., 67661., 67662., 67663., 67664., 67665.,
       67666., 67667., 67668., 67669., 67670., 67671., 67672., 67673.,
       67674., 67675., 67676., 67677., 67678., 67679., 67680., 67681

In [35]:
type(SST.coord('time').units)

cf_units.Unit

## Attempt 

### Time

In [25]:
day1 = datetime.date(1979,1,1)

In [28]:
dates = pandas.date_range(start =day1, periods = cube.data.shape[0]).to_pydatetime().tolist()

In [55]:
dates[-1]

datetime.datetime(2020, 6, 30, 0, 0)

In [37]:
toords = [idate.toordinal() for idate in dates]

In [45]:
tunit = cf_units.Unit('days since 1-1-1', calendar='gregorian')

In [48]:
time_coord = iris.coords.DimCoord(toords, standard_name='time', units=tunit)

In [49]:
time_coord

DimCoord(array([722450, 722451, 722452, ..., 737604, 737605, 737606]), standard_name='time', units=Unit('days since 1-1-1', calendar='gregorian'))

### Cube

In [50]:
new_cube = iris.cube.Cube(cube.data, units='m', var_name='Z500',
                                             dim_coords_and_dims=[(time_coord, 0),
                                                                  (cube.coord('latitude'), 1),
                                                                  (cube.coord('longitude'), 2)])
 

In [51]:
new_cube

Z500 (m),time,latitude,longitude
Shape,15157,73,144
Dimension coordinates,,,
time,x,-,-
latitude,-,x,-
longitude,-,-,x


### Grab a day

In [52]:
# Day
constraint = iris.Constraint(time=iris.time.PartialDateTime(
    day=1, year=2012, month=1))
dmy_cube = new_cube.extract(constraint)

In [53]:
dmy_cube

<iris 'Cube' of Z500 / (m) (latitude: 73; longitude: 144)>

## Write

In [54]:
iris.save(new_cube, 'NCEP-DOE_Z500.nc', zlib=True)